In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


## *Parameters*

In [0]:
# key columns list
key_col = "['flight_id']"
key_col_list = eval(key_col)

# cdc column
cdc_col = "modifiedDate"

# back-date refresh
backdated_refresh = ""

# catalog
catalog = "workspace"

# Source Object
source_object = "silver_flights"

# Source schema
source_schema = "silver"

# Target schema
target_schema = "gold"

# Target object
target_object = "DimFlights"

# Surrey key
sur_key = "Dimflightskey"


In [0]:
# key columns list
#key_col = "['airport_id']"
#key_col_list = eval(key_col)

# cdc column
#cdc_col = "modifiedDate"

# back-date refresh
#backdated_refresh = ""

# catalog
#catalog = "workspace"

# Source Object
#source_object = "silver_airports"

# Source schema
#source_schema = "silver"

# Target schema
#target_schema = "gold"

# Target object
#target_object = "DimAirports"

# Surrey key
#sur_key = "DimAirportskey"


In [0]:
# key columns list
#key_col = "['passenger_id']"
#key_col_list = eval(key_col)

# cdc column
#cdc_col = "modifiedDate"

# back-date refresh
#backdated_refresh = ""

# catalog
#catalog = "workspace"

# Source Object
#source_object = "silver_passengers"

# Source schema
#source_schema = "silver"

# Target schema
#target_schema = "gold"

# Target object
#target_object = "DimPassengers"

# Surrey key
#sur_key = "DimPassengerskey"


In [0]:
# key columns list
#key_col = "['booking_id']"
#key_col_list = eval(key_col)

# cdc column
#cdc_col = "modifiedDate"

# back-date refresh
#backdated_refresh = ""

# catalog
#catalog = "workspace"

# Source Object
#source_object = "silver_bookings"

# Source schema
#source_schema = "silver"

# Target schema
#target_schema = "gold"

# Target object
#target_object = "DimBookings"

# Surrey key
#sur_key = "DimBookingskey"


### **Incremental Data Ingestion**

#### Last Load Data

In [0]:
# no backdated refresh
if len(backdated_refresh) == 0:
  #if table EXIST in the destination 
  if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    last_load = spark.sql(f"select max({cdc_col}) from workspace.{target_schema}.{target_object}").collect()[0][0]
  
  else:
    last_load = "1900-01-01 00:00:00"
# do back data refresh
else:
  last_load = backdated_refresh


## test
last_load

datetime.datetime(2025, 7, 1, 7, 35, 46, 289000)

In [0]:
df_src = spark.sql(f"SELECT * FROM {source_schema}.{source_object} WHERE {cdc_col} > '{last_load}'")

### old vs new records

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

    # key coloumns string for incremental
    key_col_string_incremental = ", ".join(key_col_list)
    df_tgt = spark.sql(f"SELECT {key_col_string_incremental}, {sur_key}, create_date, update_date FROM {catalog}.{target_schema}.{target_object}")
else:

    # key column string initial
    key_col_string_init = [f"'' AS {i}" for i in key_col_list]
    key_col_string_init = ", ".join(key_col_string_init)
     

    df_tgt = spark.sql(f"""SELECT {key_col_string_init}, CAST('0' AS INT) AS {sur_key}, CAST('1900-01-01 00:00:00' AS timestamp) AS create_date, CAST('1900-01-01 00:00:00' AS timestamp) AS update_date WHERE 1=0""")


#### join condition

In [0]:
join_condition = ' AND '.join([f"src.{i} = tgt.{i}" for i in key_col_list])

In [0]:
df_src.createOrReplaceTempView("src")
df_tgt.createOrReplaceTempView("tgt")

df_join = spark.sql(f"""
          SELECT src.*,
                tgt.{sur_key},
                tgt.create_date,
                tgt.update_date
          FROM src
          LEFT JOIN tgt
          ON {join_condition}
          
          """)

In [0]:
df_join.display()

flight_id,airline,origin,destination,flight_date,modifiedDate,Dimflightskey,create_date,update_date
F0003,Jet Airways,East Patrickborough,North Mary,2025-07-03,2025-07-01T13:32:22.680Z,14,2025-07-01T12:34:50.022Z,2025-07-01T12:34:50.022Z
F0113,Air Asia,North Mistyberg,East Lisaton,2025-07-02,2025-07-01T13:32:22.680Z,null,null,null
F0112,Ryanair,Lake David,Port Michael,2025-07-12,2025-07-01T13:32:22.680Z,null,null,null
F0052,IndiGo,Lake Richard,Brookston,2025-06-28,2025-07-01T13:32:22.680Z,5,2025-07-01T12:34:50.022Z,2025-07-01T12:34:50.022Z
F0032,British Airways,Bushhaven,Patriciamouth,2025-07-30,2025-07-01T13:32:22.680Z,44,2025-07-01T12:34:50.022Z,2025-07-01T12:34:50.022Z
F0005,IndiGo,Bennettside,New Mistyhaven,2025-06-24,2025-07-01T13:32:22.680Z,8,2025-07-01T12:34:50.022Z,2025-07-01T12:34:50.022Z
F0111,United,South Michellemouth,Jacquelinechester,2025-07-12,2025-07-01T13:32:22.680Z,null,null,null
F0045,IndiGo,North Michaelville,Russellton,2025-07-16,2025-07-01T13:32:22.680Z,17,2025-07-01T12:34:50.022Z,2025-07-01T12:34:50.022Z


In [0]:
# old records
df_old = df_join.filter(col(f'{sur_key}').isNotNull())

# new records
df_new = df_join.filter(col(f'{sur_key}').isNull())


### preparing df_old

In [0]:
df_old_enr = df_old.withColumn('update_date', current_timestamp())

### preparing df_new

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    max_surrogate_key = spark.sql(f"""SELECT max({sur_key}) FROM {catalog}.{target_schema}.{target_object}""").collect()[0][0]
    df_new_enr = df_new.withColumn(f'{sur_key}', lit(max_surrogate_key) + lit(1) + monotonically_increasing_id())\
                    .withColumn('create_date', current_timestamp())\
                    .withColumn('update_date', current_timestamp())
else:
    max_surrogate_key = 0
    df_new_enr = df_new.withColumn(f'{sur_key}', lit(max_surrogate_key) + lit(1) + monotonically_increasing_id())\
                    .withColumn('create_date', current_timestamp())\
                    .withColumn('update_date', current_timestamp())


### ***Union of Old and New Records***

In [0]:
df_union = df_old_enr.unionByName(df_new_enr)



### ***Upsert***

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    
    dlt_obj = DeltaTable.forName(spark, f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("tgt").merge(df_union.alias("src"), f"tgt.{sur_key} = src.{sur_key}")\
                        .whenMatchedUpdateAll(condition = f"src.{cdc_col} >= tgt.{cdc_col}")\
                        .whenNotMatchedInsertAll()\
                        .execute()

else:
    df_union.write.format("delta")\
      .mode("overwrite")\
      .saveAsTable(f"{catalog}.{target_schema}.{target_object}")

In [0]:
%sql
select * from workspace.gold.dimpassengers WHERE passenger_id = 'P0049'

passenger_id,name,gender,nationality,modifiedDate,DimPassengerskey,create_date,update_date
P0049,Justin Thomas,Female,Tokelau,2025-07-01T13:32:22.664Z,7,2025-07-01T13:06:23.671Z,2025-07-01T13:52:09.891Z
